In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tensorflow.python.keras.engine.sequential import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
tf.random.set_seed(13)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install tensorflow==2.9.2

In [ ]:
!pip install keras

In [ ]:
print(tf.__version__)

In [ ]:
data_desempleo = pd.read_csv('desempleo.csv', sep=';')
print(data_desempleo.head())

In [ ]:
TargetVariable=['unemployment']
Predictors = ['minimum_salary', 'gdp', 'gfcf']
X=data_desempleo[Predictors].values
y=data_desempleo[TargetVariable].values

from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)
 
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)
 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1120, shuffle=False)


In [ ]:
def make_regression_bilstm(units_trial, Optimizer_trial):
    from keras.models import Sequential
    from keras.layers import Dense
    model = Sequential()
    model.add(Dense(units=units_trial, input_dim=3, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer=Optimizer_trial)
    return model

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

Parameter_Trials={'batch_size':[2, 4, 8, 10, 12, 14],
                    'epochs':[32, 34, 36, 60, 65, 70, 75, 80],
                    'units_trial':[2, 3, 4, 5, 6, 7, 8],
                    'Optimizer_trial':['adam', 'rmsprop']
                 }

In [ ]:
RegModel=KerasRegressor(make_regression_bilstm, verbose=0)

In [ ]:
from sklearn.metrics import make_scorer
def Accuracy_Score(orig,pred):
    MAPE = np.mean(100 * (np.abs(orig-pred)/orig))
    print('#'*70,'Accuracy:', 100-MAPE)
    return(100-MAPE)

custom_Scoring=make_scorer(Accuracy_Score, greater_is_better=True)

In [ ]:
grid_search=GridSearchCV(estimator=RegModel, 
                         param_grid=Parameter_Trials, 
                         scoring=custom_Scoring, 
                         cv=5)

In [ ]:
import time
StartTime=time.time()
grid_search.fit(X,y, verbose=1)
EndTime=time.time()
print("########## Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes')
print('### Printing Best parameters ###')
grid_search.best_params_

In [ ]:
#train the model with the best parameters
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tensorflow.python.keras.engine.sequential import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
tf.random.set_seed(13)

In [ ]:
df = pd.read_csv('desempleo.csv', sep=';')
print(df.head())

In [ ]:
train_dates = pd.to_datetime(df['date'])

In [ ]:
cols = list(df)[2:6]

In [ ]:
df_train = df[cols].astype('float')

In [ ]:
scaler = StandardScaler()
scaler.fit(df_train)
df_train_scaled = scaler.transform(df_train)

In [ ]:
X_train = []
Y_train = []
n_future = 1
n_past = 22

for i in range(n_past, len(df_train_scaled)-n_future+1):
    X_train.append(df_train_scaled[i-n_past:i, 0:df_train.shape[1]])
    Y_train.append(df_train_scaled[i+n_future-1:i+n_future, 3])

X_train, Y_train = np.array(X_train), np.array(Y_train)

In [ ]:
#GridSearchCV {'Optimizer_trial': 'adam', 'batch_size': 2, 'epochs': 34, 'units_trial': 2}
model = Sequential()
model.add(Dropout(0.05))
model.add(Bidirectional(LSTM(2, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu')))
model.add(Dropout(0.05))
model.add(Dense(Y_train.shape[1], activation='linear'))
model.compile(optimizer='adam', loss='mse')

In [ ]:
history = model.fit(X_train, Y_train, epochs=34, batch_size=2, validation_split=0.1, verbose=1)

In [ ]:
import matplotlib.pyplot as plt
plt.title('Training and validation loss')
plt.plot(history.history['loss'], label='Traininig loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()
plt.title('Training and validation loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')

In [ ]:
import datetime
n_months_for_prediction = 22
predict_periods_dates = [pd.Timestamp('2018-03-01'), pd.Timestamp('2018-04-01'), pd.Timestamp('2018-05-01'), pd.Timestamp('2018-06-01'), pd.Timestamp('2018-07-01'), pd.Timestamp('2018-08-01'), pd.Timestamp('2018-09-01'), pd.Timestamp('2018-10-01'), pd.Timestamp('2018-11-01'), pd.Timestamp('2018-12-01'), pd.Timestamp('2019-01-01'), pd.Timestamp('2019-02-01'), pd.Timestamp('2019-03-01'), pd.Timestamp('2019-04-01'), pd.Timestamp('2019-05-01'), pd.Timestamp('2019-06-01'), pd.Timestamp('2019-07-01'), pd.Timestamp('2019-08-01'), pd.Timestamp('2019-09-01'), pd.Timestamp('2019-10-01'), pd.Timestamp('2019-11-01'), pd.Timestamp('2019-12-01')]

In [ ]:
#Make Prediction
prediction = model.predict(X_train[-n_months_for_prediction:])

In [ ]:
prediction_copies = np.repeat(prediction, df_train.shape[1], axis=-1)
y_pred = scaler.inverse_transform(prediction_copies)[:,3]

In [ ]:
y_pred

In [ ]:
# Convert timestamps to date
forecast_dates = []
for time_i in predict_periods_dates:
    forecast_dates.append(time_i.date())

df_forecast = pd.DataFrame({'Date':np.array(forecast_dates), 'Unemployment rate':y_pred})
df_forecast['Date']=pd.to_datetime(df_forecast['Date'])

original = df[['date']]
date = pd.to_datetime(original['date'])
original = df[['unemployment']]
original.insert(0, 'date', date)
original = original.loc[original['date'] >= '2018-1-3']

In [ ]:
original = original.filter(["unemployment"])

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
#BiLSTM (1L)
y = np.array(original)  #predictor variable
yhat = yhat = np.array(y_pred) #predicted variable

#MSE
m_mse=mean_squared_error(y,yhat)
#MAE
mae_error = mae(y, yhat)
#MAPE
def MAPE(y,yhat):
    mape = np.mean(np.abs((y - yhat)/y))*100
    return mape
print('Test MSE: ', m_mse)
print('Test MAE: ', mae_error)
print('Test MAPE: ', MAPE(y,yhat))